In [ ]:
docker compose pull
docker compose up -d

One time setup

Option 1: Run inside the Docker container (recommended for your setup)

Since your Postgres is running inside Docker (pgvector/pgvector), you can execute psql inside that container.

List containers:

In [ ]:
docker ps

Find the one named rag_pg (from your compose file), then run:

In [ ]:
docker exec -it rag_pg psql -U postgres -d review_agent

In [ ]:
-- Enable pgvector
CREATE EXTENSION IF NOT EXISTS vector;

-- Main table (normalized fields from your CSV)
CREATE TABLE IF NOT EXISTS restaurants (
  restaurant_id TEXT PRIMARY KEY,            -- hash of Page_URL or your own ID
  name TEXT NOT NULL,
  page_url TEXT UNIQUE NOT NULL,
  star_rating NUMERIC(3,2),
  review_count INTEGER,
  categories TEXT[],                         -- '{韓国料理,居酒屋}'
  tel_reservation TEXT,
  tel TEXT,
  address TEXT,
  ward TEXT,
  area_hint TEXT,
  transportation TEXT,
  budget_dinner_min INTEGER,
  budget_dinner_max INTEGER,
  budget_lunch_min INTEGER,
  budget_lunch_max INTEGER,
  payment_methods TEXT,
  table_money TEXT,
  seats INTEGER,
  private_room BOOLEAN,
  private_use BOOLEAN,
  smoking TEXT,                              -- '禁煙'|'喫煙'|'分煙'|'加熱式のみ'|NULL
  parking BOOLEAN,
  space_facilities TEXT,
  course TEXT,
  drink TEXT,
  dishes TEXT,
  occasion TEXT,
  location_tags TEXT,
  service TEXT,
  with_children BOOLEAN,
  homepage TEXT,
  opening_day DATE,
  remarks TEXT,
  first_reviewer TEXT,
  retrieval_text_ja TEXT,                    -- concise text for embedding
  updated_at TIMESTAMP DEFAULT now()
);

-- Vectors table
CREATE TABLE IF NOT EXISTS restaurant_vectors (
  restaurant_id TEXT PRIMARY KEY REFERENCES restaurants(restaurant_id) ON DELETE CASCADE,
  embedding VECTOR(1536),                    -- adjust size to your embedding model
  updated_at TIMESTAMP DEFAULT now()
);

-- Helpful indexes
CREATE INDEX IF NOT EXISTS idx_restaurants_ward ON restaurants(ward);
CREATE INDEX IF NOT EXISTS idx_restaurants_area ON restaurants(area_hint);
CREATE INDEX IF NOT EXISTS idx_restaurants_review ON restaurants(review_count DESC);
CREATE INDEX IF NOT EXISTS idx_restaurants_smoking ON restaurants(smoking);
CREATE INDEX IF NOT EXISTS idx_restaurants_children ON restaurants(with_children);

-- Vector index (IVFFLAT). After you load data, run ANALYZE for better lists choice.
CREATE INDEX IF NOT EXISTS idx_restaurants_vec ON restaurant_vectors
USING ivfflat (embedding vector_cosine_ops) WITH (lists = 100);
\q


In [ ]:
python src/data/main_load_embed_db.py

🧨 Option B: Delete everything (including volume)

If you want to remove all data and start like brand new Postgres:

In [ ]:
docker compose down -v   # <- removes the data volume
docker compose up -d     # <- recreates a fresh Postgres instance